In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [2]:
data = pd.read_csv('dataset.csv').drop(['Unnamed: 0'],axis=1)

In [3]:
data = data.sample(frac=1).reset_index(drop=True)

In [4]:
train_data = data.head(int(225*0.8))

In [5]:
test_data = data.tail(int(225*0.2))

In [6]:
display(train_data)
display(test_data)

,Label,Tweet
0,1,"RT @nytimesworld: More than 2,500 medical work..."
1,0,@DeZurdaTeam @DiazCanelB @marti160patria @SecU...
2,0,RT @alfonslopeztena: Hundreds of Hong Kong med...
3,0,RT @frontlinepbs: As the number of cases of th...
4,0,@CNN infectious diseases typically look more s...
...,...,...
175,0,RT @ChannelNewsAsia: Hong Kong medical workers...
176,1,RT @alfonslopeztena: Hundreds of Hong Kong med...
177,0,RT @frontlinepbs: As the number of cases of th...
178,1,"RT @globalnews: More than 2,000 hospital worke..."


,Label,Tweet
181,0,@caijingxiang Best way for coronavirus prevent...
182,0,RT @annafifield: A person has died of coronavi...
183,0,RT @sehof: The censorship instructions with re...
184,0,RT @nytimes: In the middle of the coronavirus ...
185,1,RT @YohendrisQ: @DeZurdaTeam @DiazCanelB @mart...
186,0,"RT @AFP: Hong Kong medical workers strike, dem..."
187,0,RT @ReutersChina: Hong Kong reports first coro...
188,0,RT @balajis: The #coronavirus has now killed m...
189,0,"RT @QAnonNotables: Dr. Francis Boyle, drafter ..."
190,1,"RT @AFP: Hong Kong medical workers strike, dem..."
